# Notebook for taking stuttered images of candidate double stars

This notebook finds candidate double stars from wds for use in stuttered image tests. \
Then takes stuttered images of them.

Elana Urbach, Craig Lage - 07Oct22

In [ ]:
import sys, asyncio, time, os
import numpy as np
import pandas as pd
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
from astropy.time import Time, TimeDelta
from astropy import units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils.enums import RotType

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#Start classes
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(latiss.start_task, atcs.start_task)

In [ ]:
# enable components if not already done
await atcs.enable()
await latiss.enable()

### Now find candidate stars
First load in the candidate stars (preselected for being between magnitudes 6 and 9 and 9 to 200 arcsec separation).

The output dataframe contains the wds identifier, the separation between the stars in arcsec, the magnitudes of the two stars, and their RA and DEC in SkyCoord.

In [ ]:
double_stars = pd.read_csv('Double_Stars_Stuttered_Image.csv')

double_stars['skycoord'] = [SkyCoord(str(int(double_stars['RA h'][i])) + ' ' +  str(int(double_stars['RA m'][i])) + ' ' + str(double_stars['RA s'][i]), \
                            str(int(double_stars['DEC h'][i])) + ' ' +  str(int(double_stars['DEC m'][i])) + ' ' + str(double_stars['DEC s'][i]), \
                                  unit = (u.hourangle, u.deg, u.arcsec)) for i in range(len(double_stars))]

double_stars = double_stars.drop(['RA h', 'RA m', 'RA s', 'DEC h', 'DEC m', 'DEC s', 'Unnamed: 0'], axis = 1)

### Select stars

Select stars by entering the date and time.

### ENTER THE CURRENT DATE AND TIME HERE

In [ ]:
# ENTER THE CURRENT DATE AND TIME
# current_time = '2022-10-12T01:00:00'

# obstime = Time(current_time, format = 'isot', scale = 'utc')
obstime = Time.now()

In [ ]:
# Calculate altitude and azimuth based on the current time

# altAz = AltAz(obstime=vi.date.toPython(), location=AUXTEL_LOCATION)
altAz = AltAz(obstime=obstime, location=AUXTEL_LOCATION)
double_stars['obsAltAz'] = [double_stars.skycoord[i].transform_to(altAz) for i in range(len(double_stars))]
double_stars['Alt'] = [double_stars.obsAltAz[i].alt.deg for i in range(len(double_stars))]
double_stars['Az'] = [double_stars.obsAltAz[i].az.deg for i in range(len(double_stars))]

#select stars that are above 50 degrees altitude and before transit, ~ 20 degrees azimuth
candidates = double_stars.loc[double_stars.Alt > 50].loc[double_stars.Az > 15].loc[double_stars.Az < 25] 
candidates = candidates.reset_index()

In [ ]:
candidates

In [ ]:
# get RA and DEC for candidates.
for i in range(len(candidates)):
    print(candidates.index[i])
    print('RA (deg): ', candidates.skycoord[i].ra.deg)
    print('DEC (deg): ', candidates.skycoord[i].dec.deg)
    print('Theta (deg): ', candidates['Theta (deg)'][i])

### Now choose a candidate, slew to it, and take an image

Choose the index from the above list

In [ ]:
index = 0
PA = candidates['Theta (deg)'][index]
print(PA)
await atcs.slew_icrs(candidates.skycoord[index].ra.hour, candidates.skycoord[index].dec.deg, \
                     rot=PA-90.0, rot_type=RotType.PhysicalSky, offset_x = 0, offset_y = -2.5)
await latiss.take_object(exptime=2., n=1, filter='SDSSr_65mm',grating='empty_1', reason='SITCOM-468')

In [ ]:
# It may require moving the image slightly to get it positioned.
# Ideally, it should be 25 pixels (2.5 arcseconds) above the midline.
# The step below should do this if it is perfectly centered to begin with.
# Then take a new image to verify
# This may take a few iterations to get it in the right place.
# await atcs.offset_xy(y=-2.5, x=0)
# await latiss.take_object(exptime=2., n=1, filter='SDSSr_65mm',grating='empty_1', reason='SITCOM-468')

### Now take two sets of stuttered images

One set at 1" exposure and one set at 0.5 seconds.

In [ ]:
# take stuttered image
await latiss.take_stuttered(n=5, exptime=1.0, n_shift=40, row_shift=50, reason='SITCOM-468')
await latiss.take_stuttered(n=5, exptime=0.5, n_shift=40, row_shift=50, reason='SITCOM-468')

### Now repeat with a second object